Question 1: Car Matrix Generation
Under the function named generate_car_matrix write a logic that takes the dataset-1.csv as a DataFrame. Return a new DataFrame that follows the following rules:

values from id_2 as columns
values from id_1 as index
dataframe should have values from car column
diagonal values should be 0.

In [ ]:
import pandas as pd

def generate_car_matrix(dataset):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(dataset)

    # Pivot the DataFrame to create the desired matrix
    car_matrix = df.pivot(index='id_1', columns='id_2', values='car').fillna(0)

    # Set the diagonal values to 0
    car_matrix.values[[range(len(car_matrix))]*2] = 0

    return car_matrix

# Replace 'dataset-1.csv' with the actual file path
result_matrix = generate_car_matrix('dataset-1.csv')
print(result_matrix)

Question 2: Car Type Count Calculation
Create a Python function named get_type_count that takes the dataset-1.csv as a DataFrame. Add a new categorical column car_type based on values of the column car:

low for values less than or equal to 15,
medium for values greater than 15 and less than or equal to 25,
high for values greater than 25.
Calculate the count of occurrences for each car_type category and return the result as a dictionary. Sort the dictionary alphabetically based on keys.

In [ ]:
import pandas as pd

def get_type_count(dataset):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(dataset)

    # Add a new categorical column 'car_type'
    conditions = [
        (df['car'] <= 15),
        (df['car'] > 15) & (df['car'] <= 25),
        (df['car'] > 25)
    ]

    choices = ['low', 'medium', 'high']
    df['car_type'] = pd.Series(np.select(conditions, choices, default=np.nan))

    # Calculate the count of occurrences for each car_type category
    type_count = df['car_type'].value_counts().to_dict()

    # Sort the dictionary alphabetically based on keys
    type_count = {k: type_count[k] for k in sorted(type_count)}

    return type_count

# Replace 'dataset-1.csv' with the actual file path
result_type_count = get_type_count('dataset-1.csv')
print(result_type_count)

Question 3: Bus Count Index Retrieval
Create a Python function named get_bus_indexes that takes the dataset-1.csv as a DataFrame. The function should identify and return the indices as a list (sorted in ascending order) where the bus values are greater than twice the mean value of the bus column in the DataFrame.

In [ ]:
import pandas as pd

def get_bus_indexes(dataset):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(dataset)

    # Calculate the mean value of the 'bus' column
    bus_mean = df['bus'].mean()

    # Identify indices where the 'bus' values are greater than twice the mean
    bus_indexes = df[df['bus'] > 2 * bus_mean].index.tolist()

    # Sort the indices in ascending order
    bus_indexes.sort()

    return bus_indexes

# Replace 'dataset-1.csv' with the actual file path
result_bus_indexes = get_bus_indexes('dataset-1.csv')
print(result_bus_indexes)

Question 4: Route Filtering
Create a python function filter_routes that takes the dataset-1.csv as a DataFrame. The function should return the sorted list of values of column route for which the average of values of truck column is greater than 7.

In [ ]:
import pandas as pd

def filter_routes(dataset):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(dataset)

    # Calculate the average of values in the 'truck' column for each route
    avg_truck_by_route = df.groupby('route')['truck'].mean()

    # Filter routes where the average truck value is greater than 7
    filtered_routes = avg_truck_by_route[avg_truck_by_route > 7].index.tolist()

    # Sort the list of routes in ascending order
    filtered_routes.sort()

    return filtered_routes

# Replace 'dataset-1.csv' with the actual file path
result_filtered_routes = filter_routes('dataset-1.csv')
print(result_filtered_routes)

Question 5: Matrix Value Modification
Create a Python function named multiply_matrix that takes the resulting DataFrame from Question 1, as input and modifies each value according to the following logic:

If a value in the DataFrame is greater than 20, multiply those values by 0.75,
If a value is 20 or less, multiply those values by 1.25.
The function should return the modified DataFrame which has values rounded to 1 decimal place.

In [ ]:
import pandas as pd

def multiply_matrix(car_matrix):
    # Apply the specified logic to modify matrix values
    modified_matrix = car_matrix.applymap(lambda x: x * 0.75 if x > 20 else x * 1.25)

    # Round the values to 1 decimal place
    modified_matrix = modified_matrix.round(1)

    return modified_matrix

# Assuming 'result_matrix' is the DataFrame obtained from Question 1
# Replace 'result_matrix' with the actual DataFrame variable
result_matrix_modified = multiply_matrix(result_matrix)
print(result_matrix_modified)

Question 6: Time Check
You are given a dataset, dataset-2.csv, containing columns id, id_2, and timestamp (startDay, startTime, endDay, endTime). The goal is to verify the completeness of the time data by checking whether the timestamps for each unique (id, id_2) pair cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

Create a function that accepts dataset-2.csv as a DataFrame and returns a boolean series that indicates if each (id, id_2) pair has incorrect timestamps. The boolean series must have multi-index (id, id_2).

In [ ]:
import pandas as pd

def check_time_completeness(df):
    # Combine 'startDay' and 'startTime' to create a 'start_datetime' column
    df['start_datetime'] = pd.to_datetime(df['startDay'] + ' ' + df['startTime'])

    # Combine 'endDay' and 'endTime' to create an 'end_datetime' column
    df['end_datetime'] = pd.to_datetime(df['endDay'] + ' ' + df['endTime'])

    # Check if each (id, id_2) pair covers a full 24-hour period and spans all 7 days
    completeness_check = (
        df.groupby(['id', 'id_2'])
        .apply(
            lambda group: (
                (group['start_datetime'].min().time() == pd.Timestamp('00:00:00').time()) and
                (group['end_datetime'].max().time() == pd.Timestamp('23:59:59').time()) and
                (group['start_datetime'].min().day_name() == 'Monday') and
                (group['end_datetime'].max().day_name() == 'Sunday')
            )
        )
    )

    return completeness_check

# Replace 'dataset-2.csv' with the actual file path
df_dataset_2 = pd.read_csv('dataset-2.csv')
result_time_completeness = check_time_completeness(df_dataset_2)
print(result_time_completeness)

Python Task 2
Question 1: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [ ]:
import pandas as pd

def calculate_distance_matrix(dataset_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(dataset_path)

    # Create a DataFrame for distances with IDs as indices and columns
    distance_matrix = pd.DataFrame(index=df['ID'].unique(), columns=df['ID'].unique())

    # Initialize the distance matrix with zeros
    distance_matrix = distance_matrix.fillna(0)

    # Iterate over rows in the dataset to calculate cumulative distances
    for index, row in df.iterrows():
        start_id, end_id, distance = row['Start_ID'], row['End_ID'], row['Distance']

        # Update the distance matrix with bidirectional distances
        distance_matrix.at[start_id, end_id] += distance
        distance_matrix.at[end_id, start_id] += distance

    return distance_matrix

# Replace 'dataset-3.csv' with the actual file path
result_distance_matrix = calculate_distance_matrix('dataset-3.csv')
print(result_distance_matrix)

Question 2: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [ ]:
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Create an empty DataFrame to store unrolled distance values
    unrolled_distances = pd.DataFrame(columns=['id_start', 'id_end', 'distance'])

    # Iterate over the rows of the distance matrix
    for id_start in distance_matrix.index:
        for id_end in distance_matrix.columns:
            # Skip rows where id_start is equal to id_end
            if id_start == id_end:
                continue

            # Extract the distance value from the distance matrix
            distance = distance_matrix.at[id_start, id_end]

            # Append a row to the unrolled_distances DataFrame
            unrolled_distances = unrolled_distances.append({
                'id_start': id_start,
                'id_end': id_end,
                'distance': distance
            }, ignore_index=True)

    return unrolled_distances

# Assuming result_distance_matrix is the DataFrame from Question 1
# Replace result_distance_matrix with the actual DataFrame variable
result_unrolled_distances = unroll_distance_matrix(result_distance_matrix)
print(result_unrolled_distances)

Question 3: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [ ]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(df_distances, reference_value):
    # Filter rows with the specified reference value as id_start
    reference_rows = df_distances[df_distances['id_start'] == reference_value]

    # Calculate the average distance for the reference value
    average_distance = reference_rows['distance'].mean()

    # Calculate the 10% threshold
    threshold_low = 0.9 * average_distance
    threshold_high = 1.1 * average_distance

    # Filter rows within the 10% threshold
    filtered_rows = df_distances[
        (df_distances['distance'] >= threshold_low) &
        (df_distances['distance'] <= threshold_high)
    ]

    # Get unique values from the id_start column and sort the list
    result_ids = sorted(filtered_rows['id_start'].unique().tolist())

    return result_ids

# Assuming result_unrolled_distances is the DataFrame from Question 2
# Replace result_unrolled_distances with the actual DataFrame variable
result_reference_ids = find_ids_within_ten_percentage_threshold(result_unrolled_distances, reference_value=1)
print(result_reference_ids)

Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

0.8 for moto
1.2 for car
1.5 for rv
2.2 for bus
3.6 for truck

In [ ]:
import pandas as pd

def calculate_toll_rate(df_distances):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Add new columns for each vehicle type
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        df_distances[vehicle_type] = df_distances['distance'] * rate_coefficient

    return df_distances

# Assuming result_unrolled_distances is the DataFrame from Question 2
# Replace result_unrolled_distances with the actual DataFrame variable
result_distances_with_toll = calculate_toll_rate(result_unrolled_distances)
print(result_distances_with_toll)

Question 5: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
start_time and end_time must be of type datetime.time() with the values from time range given below.
Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
Weekends (Saturday and Sunday):

Apply a constant discount factor of 0.7 for all times.
For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [ ]:
import pandas as pd
import numpy as np

def calculate_time_based_toll_rates(df):
    # Define time ranges and discount factors for weekdays and weekends
    weekday_time_ranges = [
        ('00:00:00', '10:00:00', 0.8),
        ('10:00:00', '18:00:00', 1.2),
        ('18:00:00', '23:59:59', 0.8)
    ]
    weekend_discount_factor = 0.7

    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame(columns=['id_start', 'id_end', 'start_day', 'start_time', 'end_day', 'end_time'])

    # Iterate over unique (id_start, id_end) pairs
    for (id_start, id_end), group_df in df.groupby(['id_start', 'id_end']):
        # Ensure each pair covers a full 24-hour period and spans all 7 days of the week
        unique_days = pd.to_datetime(group_df['start_day']).dt.day_name().unique()
        unique_times = pd.to_datetime(group_df['start_time']).dt.time.unique()

        if len(unique_days) == 7 and len(unique_times) == 1:
            # Assign values for each day of the week and time range
            for day in unique_days:
                for start_time, end_time, discount_factor in weekday_time_ranges:
                    start_datetime = pd.to_datetime('2023-01-01 ' + start_time)
                    end_datetime = pd.to_datetime('2023-01-01 ' + end_time)

                    result_df = result_df.append({
                        'id_start': id_start,
                        'id_end': id_end,
                        'start_day': day,
                        'start_time': start_datetime.time(),
                        'end_day': day,
                        'end_time': end_datetime.time(),
                        'discount_factor': discount_factor
                    }, ignore_index=True)
        else:
            # Assign values for weekends
            for day in unique_days:
                start_datetime = pd.to_datetime('2023-01-01 00:00:00')
                end_datetime = pd.to_datetime('2023-01-01 23:59:59')

                result_df = result_df.append({
                    'id_start': id_start,
                    'id_end': id_end,
                    'start_day': day,
                    'start_time': start_datetime.time(),
                    'end_day': day,
                    'end_time': end_datetime.time(),
                    'discount_factor': weekend_discount_factor
                }, ignore_index=True)

    return result_df

# Assuming result_unrolled_distances is the DataFrame from Question 3
# Replace result_unrolled_distances with the actual DataFrame variable
result_time_based_toll_rates = calculate_time_based_toll_rates(result_unrolled_distances)
print(result_time_based_toll_rates)